# **DAILY MACRO RELATIVE VALUE TRADE FRAMEWORK**
### Author:  Deborah Akintoye
#### **Purpose:** Convert one macro headline into a backtestable relative-value trade with clear horizon, factor sensitivities, execution logic, and weekly performance tracking.

---


## 1. Trade Metadata & Idea Structuring

**Date:    12-Jan-2026**

**News Source / Catalyst:** "US prosecutors launch criminal investigation into Federal Reserve’s Jay Powell" - Claire Jones in London and Kate Duguid in New York

**Markets (s) Impacted:** FX, EQ, COM, FI (but, trade will be executed via FX and COM)

**Trade Taxonomy:** RV Macro / Structural FX

**Trade Idea (1 sentence):**
> The Trump administration's litigation against the Fed, commands a short dollar trade against a basket of European currencies and safe-haven assets.

**Client/Desk Pitch (30 seconds):**
> There has been rising concerns over Trump's agressive approach to forcing the hand of the Federal Reserve in favour of lower rates, however the recent subpoenas marks a global turning point. This issue can be analysed from multiple angles, which all result in a high-conviction short on USD. Markets will start to price in greater inflation expectations, inflation risk premium, and term premium if the Fed's independence comes under further attack. Whilst Powell has remained strong in his Fed independence, his term as chairman will be coming to an end in May, with his-expected replacements cozing up to the White House. 

**Why Now:**
- The is the first wave of litigation by the White House against the Fed, allowing this trade to crossover from thematic into event-driven
- There is limited geopolitical risk felt amongst our European currency-basket, when juxtaposed to the US, making this trade FX fundamentally driven
- Asia has already shown to be pricing in this risk with gold hitting USD 4,600 a troy ounce and other bullions rallying, all prior to US and Europe-market opening, creating slight-information asymmetry to be traded on

---

## 2. Macro Hypothesis & Instruments

### **Macro Hypothesis:**
- The value of USD will trade down against their European peers, out of fears driving by a lack of separation between the government and central bank
- This lack of value will increase the risk premium found amongst UST, USD, inflating US yields
- Taking a longer view, as Powell is stepping down in May, and will likely be replaced by a more dovish peer, EQ investors will hold off in overly-participating in trade due to the increased likelihood of lower rates in the future, and FI investors will reduce increase US new issue participation on the likelihood of locking in higher rates prior, to the Fed's May pivot

**Positioning (Tickers + Rationale):**
- EUR/USD (EURUSD=X), GBP/USD (GBPUSD=X), NOK/USD (NOKUSD=X), Gold (GC=F)

    **Weights:** Equal-Weights
---

## 3. Factor Sensitivity & Horizon Alignment

| Factor                | Strength (S/M/W) | Notes |
|----------------------|------------------|-------|
| USD / FX             |          S        |       |
| Oil Beta             |          W        |       |
| Rates Duration       |           S       |       |
| Growth vs Value      |             W     |       |
| Risk-On/Off          |            M      |       |
| China/EM Beta        |             W     |       |
| Volatility Regime    |            S      |       |

**Catalyst Type:** Policy + Regime + Event Driven

**Realisation Window:** 1-week (to take quick volatility driven profit, however could be extended depending on client's manadate)  

**Holding Period Selected:**  10-trading days (as the White House is expected to announce their chairman nomination in January 2026)

**Alignment Score:** Good

---


## 4. Risk Controls & Stop Logic

**Position Sizing:** N/A
**Stop-Loss:** (portfolio-level %) 0.5% (0.005)  
**Take-Profit (optional):**  N/A
**Re-entry:** Yes/No  
**FX-Hedging Required:** No  
**Liquidity Notes:** N/A

**Failure Mode (tick):**
- [ ] Time
- [ ] Price
- [ ] Information

---


# **PYTHON IMPLEMENTATION**

In [1]:
# --- Python Imports ---

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

pd.set_option('display.float_format', lambda x: f'{x:.4f}')


In [26]:
# --- Parameters ---

long_tickers = ["EURUSD=X", "GBPUSD=X", "NOKUSD=X", "GC=F"]
short_tickers = ["UST"] # Using this ticker to reduce python errors                   

start_date = "2020-01-01"                        
holding_period = 10                              
stop_loss = -0.005                                
take_profit = 0.02                              

# --- Weightings ---

weights_long = 1 / len(long_tickers)
weights_short = 1 / len(short_tickers)


In [27]:
# --- Data Download ---

tickers = long_tickers + short_tickers
data = yf.download(tickers, start=start_date)["Close"]

# Drop empty cols & make sure no NA

data = data.dropna(how='all').ffill()

returns = data.pct_change().dropna()


/var/folders/nn/rjk_sthj67g1rcv7qxd8_14m0000gn/T/ipykernel_35607/911793947.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, start=start_date)["Close"]
[*********************100%***********************]  5 of 5 completed


In [19]:
# --- Spread Returns ---

long_returns = returns[long_tickers].mean(axis=1)
short_returns = returns[short_tickers].mean(axis=1)

strategy_returns = long_returns - short_returns


In [28]:
# --- Backtest Simulation ---

trade_pnl = []
trade_dates = []

dates = strategy_returns.index
i = 0
n = len(strategy_returns)

while i < n - holding_period:
    pnl = 0.0
    stopped = False
    entry_date = dates[i]
    
    for j in range(holding_period):
        daily_ret = strategy_returns.iloc[i + j]
        pnl = (1 + pnl) * (1 + daily_ret) - 1
        
        if pnl <= stop_loss:
            trade_pnl.append(pnl)
            trade_dates.append(entry_date)
            stopped = True
            break
        
        if take_profit is not None and pnl >= take_profit:
            trade_pnl.append(pnl)
            trade_dates.append(entry_date)
            stopped = True
            break
    
    if not stopped:
        trade_pnl.append(pnl)
        trade_dates.append(entry_date)
    
    i += holding_period

trade_results = pd.DataFrame({
    "Trade Date": trade_dates,
    "PnL": trade_pnl
})

trade_results.tail()


,Trade Date,PnL
151,2025-10-21,-0.0185
152,2025-11-04,-0.0064
153,2025-11-18,-0.0132
154,2025-12-02,-0.0054
155,2025-12-16,-0.0094


In [29]:
# --- Performance Summary ---

summary = {
    "Total Trades": len(trade_results),
    "Win Rate": (trade_results["PnL"] > 0).mean(),
    "Average PnL": trade_results["PnL"].mean(),
    "Best Trade": trade_results["PnL"].max(),
    "Worst Trade": trade_results["PnL"].min(),
    "Stop-Out Frequency": (trade_results["PnL"] <= stop_loss).mean()
}

pd.Series(summary)


Total Trades         156.0000
Win Rate               0.4167
Average PnL            0.0028
Best Trade             0.0372
Worst Trade           -0.0324
Stop-Out Frequency     0.5769
dtype: float64

## Interpretation & Trader Notes

**Absolute Metrics:**
- Total Trades: 156.0
- Win Rate: 0.4167
- Avg PnL: 0.0028
- Best: 0.0372
- Worst: -0.0324
- Stop-Outs: 0.5769

**What went right:**
- 

**What went wrong:**
- 

**Factor that actually drove results:**
- 

**Would I trade this live?** (Yes / No and why)
>

---

# **BLOOMBERG-STYLE TRADE BLOTTER**

In [30]:
import uuid
from datetime import datetime

def generate_trade_id(prefix="RV"):
    return f"{prefix}-{uuid.uuid4().hex[:8].upper()}"


In [31]:
# --- Computing Synthetic Basket Price Series ---

basket_long = (data[long_tickers] * weights_long).sum(axis=1)
basket_short = (data[short_tickers] * weights_short).sum(axis=1)

basket_spread = basket_long - basket_short


In [32]:
# --- Backtesting Loop Modified for Blotter ---

blotter_rows = []

i = 0
n = len(strategy_returns)
dates = strategy_returns.index

while i < n - holding_period:
    entry_date = dates[i]
    exit_date = dates[min(i + holding_period - 1, n - 1)]
    
    entry_price = basket_spread.loc[entry_date]
    
    pnl = 0.0
    stopped = False
    trade_status = "CLOSED"
    
    for j in range(holding_period):
        daily_ret = strategy_returns.iloc[i + j]
        pnl = (1 + pnl) * (1 + daily_ret) - 1
        
        if pnl <= stop_loss:
            exit_date = dates[i + j]
            trade_status = "STOPPED"
            stopped = True
            break
    
    if not stopped and take_profit is not None and pnl >= take_profit:
        exit_date = dates[i + j]
        trade_status = "TAKE_PROFIT"
    
    exit_price = basket_spread.loc[exit_date]
    holding_days = (exit_date - entry_date).days
    
    row = {
        "TradeID": generate_trade_id(prefix="RV"),
        "Timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "Strategy": "Macro RV Spread",
        "Taxonomy": trade_taxonomy if 'trade_taxonomy' in globals() else "RV Macro Spread",
        "LongLeg": ", ".join(long_tickers),
        "ShortLeg": ", ".join(short_tickers),
        "EntryDate": entry_date.strftime("%Y-%m-%d"),
        "ExitDate": exit_date.strftime("%Y-%m-%d"),
        "EntryPrice": entry_price,
        "ExitPrice": exit_price,
        "GrossPnL(%)": pnl * 100,
        "StopLoss(%)": stop_loss * 100,
        "TakeProfit(%)": take_profit * 100 if take_profit else None,
        "Status": trade_status,
        "HoldingPeriod": holding_days,
        "Notes": "",  # Optional notes field
    }
    blotter_rows.append(row)
    
    i += holding_period

blotter = pd.DataFrame(blotter_rows)
blotter.tail()


,TradeID,Timestamp,Strategy,Taxonomy,LongLeg,ShortLeg,EntryDate,ExitDate,EntryPrice,ExitPrice,GrossPnL(%),StopLoss(%),TakeProfit(%),Status,HoldingPeriod,Notes
151,RV-B1426ECF,2026-01-12 07:17:53,Macro RV Spread,RV Macro Spread,"EURUSD=X, GBPUSD=X, NOKUSD=X, GC=F",UST,2025-10-21,2025-10-21,977.9090,977.9090,-1.8497,-0.5000,2.0000,STOPPED,0,
152,RV-EB08EF9A,2026-01-12 07:17:53,Macro RV Spread,RV Macro Spread,"EURUSD=X, GBPUSD=X, NOKUSD=X, GC=F",UST,2025-11-04,2025-11-04,943.6689,943.6689,-0.6413,-0.5000,2.0000,STOPPED,0,
153,RV-684D183C,2026-01-12 07:17:53,Macro RV Spread,RV Macro Spread,"EURUSD=X, GBPUSD=X, NOKUSD=X, GC=F",UST,2025-11-18,2025-11-20,972.1107,970.7095,-1.3177,-0.5000,2.0000,STOPPED,2,
154,RV-2C4F83B2,2026-01-12 07:17:53,Macro RV Spread,RV Macro Spread,"EURUSD=X, GBPUSD=X, NOKUSD=X, GC=F",UST,2025-12-02,2025-12-02,1003.1512,1003.1512,-0.5353,-0.5000,2.0000,STOPPED,0,
155,RV-F90DDDD1,2026-01-12 07:17:53,Macro RV Spread,RV Macro Spread,"EURUSD=X, GBPUSD=X, NOKUSD=X, GC=F",UST,2025-12-16,2025-12-18,1032.9500,1041.4723,-0.9374,-0.5000,2.0000,STOPPED,2,


In [33]:
blotter.to_csv("trade_blotter.csv", index=False)